#TASK1. 데이터 전처리

In [3]:
import pandas as pd
data= pd.read_csv("dataset_emotion.txt", sep=";", header=None, names=["text", "label"])
data.head()

,text,label
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness


In [4]:
data= data.drop_duplicates()
print(data.shape)

(17999, 2)


중복된 데이터를 제거함. 총 17999개의 데이터가 남아있음. 그 외 결측치는 존재하지 않음.

#불용어 제거

In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab') # Download the missing resource
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [6]:
def remove_stopwords(text):
  words = word_tokenize(text.lower())
  filtered = [word for word in words if word not in stop_words]
  return ' '.join(filtered)

In [7]:
data1 = data['text'].apply(remove_stopwords)
data1.head()
data['text_removed'] = data1
data= data[['text_removed', 'label']]
data.head()

,text_removed,label
0,im feeling rather rotten im ambitious right,sadness
1,im updating blog feel shitty,sadness
2,never make separate ever want feel like ashamed,sadness
3,left bouquet red yellow tulips arm feeling sli...,joy
4,feeling little vain one,sadness


stopwords에 저장된 단어들이 잘 제거됐음을 확인할 수 있다.

#텍스트 데이터 인코딩(CountVectorizer, tf-idf)

**CountVectorizer**

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X_cv = vectorizer.fit_transform(data['text_removed'])
print(X_cv.shape)

(17999, 16015)


In [9]:
X_cv[3].toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

17999개의 문장과 16015개의 고유단어가 존재함을 알 수 있음. (생각보다 고유단어 수가 적어 더블체크).

**tf-idf**

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(data['text_removed'])

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(data['label'])
data['label_encoded'] = y
data

,text_removed,label,label_encoded
0,im feeling rather rotten im ambitious right,sadness,4
1,im updating blog feel shitty,sadness,4
2,never make separate ever want feel like ashamed,sadness,4
3,left bouquet red yellow tulips arm feeling sli...,joy,2
4,feeling little vain one,sadness,4
...,...,...,...
17995,brief time beanbag said anna feel like beaten,sadness,4
17996,turning feel pathetic still waiting tables sub...,sadness,4
17997,feel strong good overall,joy,2
17998,feel like rude comment im glad,anger,0


'anger':0

'fear':1

'joy':2

'love':3

'sadness':4

'surprise':5

뚜렷한 감정이 드러나지 않을 때 -> neutral로 처리

꼭 love라는 단어가 포함되었다고 해서 그 감정이 아닐 수 있음(널 사랑하지 않아, 슬픔은 사치)-> 단어 간 문맥 파악 필요

너무 다양한 감정이 존재함

#TASK2. 피쳐 엔지니어링

감정분석의 경우 특수문자로 피처 엔지니어링을 진행할 수 있으나 해당 데이터에는 특수문자가 존재하지 않음..

단어 간 문맥을 반영하기 위해 CPU 환경의 word2vec 활용.

In [12]:
!pip install gensim

In [13]:
sentences = [text.split() for text in data['text_removed']]
from gensim.models import Word2Vec
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

sentence에 단어 리스트를 저장해두고 word2vec에 학습시킨다. 문맥으로 고려할 주변 단어의 범위는 5로 지정했다. 각 단어마다 벡터 생성.

In [15]:
import numpy as np

def sentence_vector(sentence, model):
    words = sentence.split()
    vecs = [model.wv[word] for word in words if word in model.wv]
    return np.mean(vecs, axis=0)
X_w2v = np.array([sentence_vector(text, w2v_model) for text in data['text_removed']])

이를 문장으로 확장. 문장 내 단어 벡터들의 평균값을 문장 벡터로 산출

텍스트 데이터를 수치형 피처로 변환.

모델에 입력하기 위해 Word2Vec을 활용하여 문장 단위 벡터를 생성함. 각 문장이 벡터로 표현되었으며 문맥 정보가 반영되게 함.